In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

# تنظیمات اتصال به SQL Server با Windows Authentication
params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-NI8V1U6\\SQL2022;"
    "DATABASE=OlistStaging;"
    "Trusted_Connection=yes;"
)

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
print("✅ اتصال با موفقیت برقرار شد.")


In [ ]:
def load_csv_to_sql(file_path, table_name, engine, if_exists='append'):
    df = pd.read_csv(file_path)
    df.to_sql(table_name, con=engine, index=False, if_exists=if_exists)
    print(f"📥 داده‌ها به جدول '{table_name}' منتقل شدند. (تعداد ردیف: {len(df)})")


In [ ]:
base_path = '../data/processed/'

load_csv_to_sql(base_path + 'olist_customers_cleaned.csv', 'Customers', engine)
load_csv_to_sql(base_path + 'olist_order_items_cleaned.csv', 'OrderItems', engine)
load_csv_to_sql(base_path + 'olist_order_payments_cleaned.csv', 'Payments', engine)
load_csv_to_sql(base_path + 'olist_order_reviews_cleaned.csv', 'Reviews', engine)
load_csv_to_sql(base_path + 'olist_products_cleaned.csv', 'Products', engine)
load_csv_to_sql(base_path + 'olist_sellers_cleaned.csv', 'Sellers', engine)
